
# Tutorial 04: Extração de Dados em JSON

**Objetivos de Aprendizagem:**
- Aprender a usar `response_format={'type': 'json_object'}` para estruturar respostas
- Extrair dados estruturados de texto livre
- Processar imagens de documentos e extrair informações
- Transcrever áudio e converter para JSON estruturado
- Extrair dados de PDFs e normalizar em JSON

**Pré-requisitos:**
- Tutorial 01: Conversa Básica concluído
- Tutorial 02: Agentes Especializados (recomendado)
- Compreensão básica de JSON e estruturas de dados

## Introdução

A extração de dados estruturados é uma das aplicações mais poderosas da API de Chat Completions. Com o parâmetro `response_format={'type': 'json_object'}`, podemos transformar qualquer tipo de conteúdo (texto, imagens, áudio, PDFs) em dados estruturados em JSON.

### Por que Extrair Dados em JSON?

1. **Integração com Sistemas**: JSON é o formato padrão para APIs e bancos de dados
2. **Automação**: Permite processar documentos automaticamente
3. **Normalização**: Padroniza dados de diferentes fontes
4. **Validação**: Estrutura facilita validação e tratamento de erros
5. **Processamento**: Facilita manipulação e armazenamento de dados

### Conceitos-Chave

**response_format:**
- `{'type': 'json_object'}` força a resposta a ser JSON válido
- O modelo sempre retorna JSON, mesmo que o prompt não mencione
- **Importante**: O prompt DEVE mencionar "JSON" explicitamente
- Garante estrutura consistente e previsível

**Casos de Uso Comuns:**
- Extração de leads de textos ou formulários
- Processamento de documentos financeiros (extratos, boletos)
- Transcrição e estruturação de áudios
- Normalização de dados de PDFs
- OCR e extração de dados de imagens
- Processamento de e-mails e mensagens
- Estruturação de dados de planilhas

**Fluxo de Processamento:**
1. Receber conteúdo (texto, imagem, áudio, PDF)
2. Processar/transcrever se necessário (Whisper para áudio, OCR para imagens)
3. Enviar para API com schema JSON definido no prompt
4. Receber e validar JSON retornado
5. Armazenar ou processar dados estruturados

**Modelos Recomendados:**
- `gpt-4o-mini`: Otimizado para tarefas estruturadas, mais econômico
- `gpt-4o`: Maior precisão para documentos complexos
- `gpt-4-turbo`: Boa alternativa para tarefas de extração

In [1]:

# Configuração inicial
import base64
import json
import os
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader

# Carregar variáveis de ambiente
load_dotenv()

# Inicializar cliente OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("Cliente OpenAI configurado com sucesso!")

Cliente OpenAI configurado com sucesso!


## Função Auxiliar: chat_to_json()

Vamos criar uma função auxiliar que simplifica a extração de dados em JSON. Esta função será usada em todos os exemplos.

**Como funciona:**
- Recebe um prompt do sistema e conteúdo do usuário
- Configura `response_format={'type': 'json_object'}` automaticamente
- Retorna os dados já parseados como dicionário Python
- Usa `gpt-4o-mini` (otimizado para tarefas estruturadas e mais econômico)


In [2]:
# Função auxiliar para extrair dados em JSON
def chat_to_json(system_prompt: str, user_content):
    """
    Extrai dados estruturados em JSON usando a API de Chat Completions.
    
    Args:
        system_prompt: Prompt do sistema definindo o contexto e comportamento
        user_content: Conteúdo do usuário (texto, imagem, etc.)
    
    Returns:
        dict: Dados extraídos em formato JSON (já parseado)
    """
    completion = client.chat.completions.create(
        model='gpt-4o-mini',  # Modelo otimizado para tarefas estruturadas
        response_format={'type': 'json_object'},  # Força resposta em JSON válido
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_content},
        ],
    )
    # Parse do JSON retornado
    return json.loads(completion.choices[0].message.content)

print("Função chat_to_json() definida e pronta para uso!")


Função chat_to_json() definida e pronta para uso!



## Exemplo 1: Extrair Dados de Texto Livre

O caso mais simples: extrair informações estruturadas de um texto digitado pelo usuário.

**Quando usar:**
- Processar formulários de contato
- Extrair informações de mensagens ou e-mails
- Estruturar dados de chat ou conversas
- Normalizar informações de diferentes formatos

**Vantagens:**
- Não requer processamento prévio
- Funciona com qualquer texto
- Rápido e eficiente
- Ideal para automação de processos

**Importante:**
- O prompt DEVE mencionar "JSON" explicitamente
- Defina claramente o schema desejado (chaves e tipos)
- Use `null` para campos ausentes quando apropriado

In [3]:

# Definir o schema JSON desejado
lead_schema_description = (
    'Retorne apenas JSON com as chaves name, email, phone e company. '
    'Use null quando algum dado não aparecer.'
)

# Texto de exemplo (poderia vir de um formulário, email, chat, etc.)
text_sample = (
    'Olá, me chamo Beatriz Martins (beatriz.martins@example.com) e trabalho na Orion Data. '
    'Meu telefone comercial é (21) 98877-6655.'
)

print("Texto original:")
print(text_sample)
print()
print("Extraindo dados estruturados...")
print()

# Extrair dados em JSON
lead_data = chat_to_json(
    system_prompt='Você extrai campos de leads e responde somente JSON válido.',
    user_content=f"{lead_schema_description}\n\nTexto de origem:\n{text_sample}"
)

print("Dados extraídos em JSON:")
print(json.dumps(lead_data, indent=2, ensure_ascii=False))


Texto original:
Olá, me chamo Beatriz Martins (beatriz.martins@example.com) e trabalho na Orion Data. Meu telefone comercial é (21) 98877-6655.

Extraindo dados estruturados...

Dados extraídos em JSON:
{
  "name": "Beatriz Martins",
  "email": "beatriz.martins@example.com",
  "phone": "(21) 98877-6655",
  "company": "Orion Data"
}



## Exemplo 2: Extrair Dados de Imagens (OCR com Vision)

Usa a capacidade multimodal do GPT-4 para ler e extrair dados de imagens de documentos. Ideal para processar comprovantes, extratos bancários, recibos, etc.

**Quando usar:**
- Processar documentos escaneados
- Extrair dados de comprovantes e recibos
- Digitalizar formulários preenchidos
- OCR de documentos financeiros

**Requisitos:**
- Modelo com suporte a visão (gpt-4o, gpt-4o-mini, gpt-4-turbo)
- Imagem em formato base64 ou URL

**Vantagens:**
- Não precisa de OCR externo
- Entende contexto visual (layout, posição)
- Pode processar documentos complexos
- Extração inteligente de campos

**Processo:**
1. Converter imagem para base64
2. Criar mensagem multimodal (texto + imagem)
3. Definir schema JSON no prompt
4. Extrair dados estruturados

In [7]:

# Caminho da imagem (certifique-se de que o arquivo existe)
image_path = Path('assets/extrato_bancario.png')

# Verificar se o arquivo existe
if not image_path.exists():
    print(f"Arquivo não encontrado: {image_path}")
    print("   Crie uma imagem de exemplo ou ajuste o caminho.")
else:
    # Converter imagem para base64
    image_base64 = base64.b64encode(image_path.read_bytes()).decode('utf-8')
    
    print(f"Processando imagem: {image_path.name}")
    print()
    
    # Criar mensagem multimodal (texto + imagem)
    image_message = [
        {
            'type': 'text',
            'text': 'Extraia emissor, número do documento, total e data de vencimento. Responda apenas JSON '
                    'com as chaves issuer, document_number, total, due_date. Use null para campos ausentes.'
        },
        {
            'type': 'image_url',
            'image_url': {
                'url': f'data:image/png;base64,{image_base64}'
            }
        }
    ]
    
    # Extrair dados da imagem
    image_data = chat_to_json(
        system_prompt='Você é responsável por leitura de documentos financeiros e devolve somente JSON válido.',
        user_content=image_message
    )
    
    print("Dados extraídos da imagem:")
    print(json.dumps(image_data, indent=2, ensure_ascii=False))


Processando imagem: extrato_bancario.png

Dados extraídos da imagem:
{
  "issuer": "Banco Orion",
  "document_number": "1154-5/7880",
  "total": "13.700,00",
  "due_date": "02/10/2025"
}



## Exemplo 3: Extrair Dados de Áudio (Speech-to-Text + JSON)

Primeiro transcreve o áudio com Whisper, depois normaliza para JSON estruturado.

**Quando usar:**
- Processar chamadas de atendimento
- Cadastros por voz
- Transcrição de reuniões
- Processamento de mensagens de voz

**Processo:**
1. Transcrever áudio com Whisper API
2. Extrair texto transcrito
3. Processar texto com Chat Completions para JSON
4. Estruturar dados conforme schema

**Vantagens:**
- Combina precisão do Whisper com estruturação do GPT
- Processa áudios em diferentes idiomas
- Extrai informações mesmo de áudios com ruído
- Ideal para automação de atendimento

In [9]:

# Caminho do arquivo de áudio
audio_path = Path('assets/cadastro_cliente.wav')

# Verificar se o arquivo existe
if not audio_path.exists():
    print(f"Arquivo não encontrado: {audio_path}")
    print("   Crie um arquivo de áudio de exemplo ou ajuste o caminho.")
else:
    print(f"Processando áudio: {audio_path.name}")
    print()
    
    # Passo 1: Transcrever áudio com Whisper
    print("1. Transcrevendo áudio com Whisper...")
    transcription = client.audio.transcriptions.create(
        model='whisper-1',
        file=audio_path.open('rb'),
        language='pt'  # Especificar português melhora precisão
    )
    
    print(f"   Transcrição: {transcription.text[:100]}...")
    print()
    
    # Passo 2: Extrair dados estruturados do texto transcrito
    print("2. Extraindo dados estruturados...")
    audio_data = chat_to_json(
        system_prompt='Você transforma pedidos por voz em cadastros estruturados e responde somente JSON válido.',
        user_content=(
            'Transforme a transcrição a seguir em JSON estruturado com as chaves name, email, phone e address. '
            'Use null para campos ausentes.\n\nTranscrição:\n'
            + transcription.text
        )
    )
    
    print()
    print("Dados extraídos do áudio:")
    print(json.dumps(audio_data, indent=2, ensure_ascii=False))


Processando áudio: cadastro_cliente.wav

1. Transcrevendo áudio com Whisper...
   Transcrição: Olá, aqui é Beatriz Martins falando. Estou abrindo meu cadastro completo. E meu e-mail principal é B...

2. Extraindo dados estruturados...

Dados extraídos do áudio:
{
  "name": "Beatriz Martins",
  "email": "Beatriz.Martins@arrobaexample.com",
  "phone": "119-888-15442",
  "address": "Rua das Acácias, número 240, apartamento 32, bairro Jardim Aurora, São Paulo, capital NP, OR"
}



## Exemplo 4: Extrair Dados de PDFs

Extrai texto de PDFs e normaliza em JSON estruturado.

**Quando usar:**
- Processar fichas cadastrais
- Extrair dados de contratos
- Digitalizar formulários em PDF
- Normalizar documentos legais

**Processo:**
1. Extrair texto do PDF usando biblioteca (pypdf, pdfplumber)
2. Processar texto com Chat Completions
3. Estruturar dados conforme schema JSON
4. Validar e armazenar dados

**Vantagens:**
- Processa PDFs não editáveis
- Entende layout e estrutura
- Extrai informações mesmo de PDFs escaneados (com OCR prévio)
- Normaliza dados de diferentes formatos de PDF

**Limitações:**
- PDFs com imagens precisam de OCR prévio
- PDFs muito complexos podem precisar de processamento adicional
- Qualidade depende da extração de texto do PDF

In [11]:

# Caminho do arquivo PDF
pdf_path = Path('assets/ficha_cadastro.pdf')

# Verificar se o arquivo existe
if not pdf_path.exists():
    print(f"Arquivo não encontrado: {pdf_path}")
    print("   Crie um arquivo PDF de exemplo ou ajuste o caminho.")
else:
    print(f"Processando PDF: {pdf_path.name}")
    print()
    
    # Passo 1: Extrair texto do PDF
    print("1. Extraindo texto do PDF...")
    reader = PdfReader(str(pdf_path))
    pdf_text = '\n'.join(
        page.extract_text().strip()
        for page in reader.pages
        if page.extract_text()
    )
    
    print(f"   Texto extraído ({len(pdf_text)} caracteres)")
    print(f"   Preview: {pdf_text[:100]}...")
    print()
    
    # Passo 2: Definir schema e extrair dados
    print("2. Extraindo dados estruturados...")
    pdf_instruction = (
        'A seguir está o texto extraído de um PDF de ficha cadastral. '
        'Responda apenas JSON com as chaves name, document_id, birth_date e city. '
        'Use null quando algum campo não estiver presente.'
    )
    
    pdf_data = chat_to_json(
        system_prompt='Você normaliza fichas cadastrais em JSON válido.',
        user_content=f"{pdf_instruction}\n\nConteúdo extraído do PDF:\n{pdf_text}"
    )
    
    print()
    print("Dados extraídos do PDF:")
    print(json.dumps(pdf_data, indent=2, ensure_ascii=False))


Processando PDF: ficha_cadastro.pdf

1. Extraindo texto do PDF...
   Texto extraído (793 caracteres)
   Preview: Orion HR Solutions
 Ficha Cadastral de Colaborador
Dados pessoais
Nome completo:
Beatriz Martins da ...

2. Extraindo dados estruturados...

Dados extraídos do PDF:
{
  "name": "Beatriz Martins da Silva",
  "document_id": "123.456.789-00",
  "birth_date": "1990-03-14",
  "city": "São Paulo"
}


## Boas Práticas

### 1. Validação de JSON
- Sempre valide o JSON antes de gravar em banco
- Use bibliotecas como `jsonschema` para validação
- Trate erros de parsing (JSON inválido)
- Implemente fallbacks para casos de falha

### 2. Schemas Consistentes
- Mantenha schemas específicos por domínio
- Documente os schemas esperados
- Versionar schemas quando necessário
- Use tipos consistentes (datas, números, strings)

### 3. Tratamento de Erros
- Trate arquivos não encontrados
- Valide tamanho de arquivos antes de processar
- Implemente retry para falhas temporárias
- Log de erros para debugging

### 4. Otimização e Performance
- Use modelos apropriados (gpt-4o-mini para tarefas simples)
- Limite tamanho de arquivos processados
- Cache resultados quando possível
- Processe em lote quando apropriado

### 5. Segurança e Privacidade
- Não armazene dados sensíveis desnecessariamente
- Limpe uploads temporários após processamento
- Valide e sanitize dados extraídos
- Respeite LGPD/GDPR ao processar dados pessoais

### 6. Monitoramento
- Registre hashes dos arquivos processados (evitar duplicatas)
- Monitore uso de tokens e custos
- Acompanhe taxa de sucesso de extração
- Versionar prompts para rastreabilidade

## Exercícios Práticos

### Exercício 1: Extrair Dados de E-mail
Crie um sistema que extrai informações de e-mails de contato (nome, assunto, mensagem, contato) e estrutura em JSON.

### Exercício 2: Processar Múltiplos Documentos
Crie uma função que processa uma pasta de PDFs e extrai dados de todos, retornando uma lista de JSONs.

### Exercício 3: Validação e Tratamento de Erros
Implemente validação de schema JSON e tratamento de erros robusto para um sistema de extração de dados.

## Referências

- [Documentação - Chat Completions](https://platform.openai.com/docs/guides/text-generation)
- [Documentação - Vision API](https://platform.openai.com/docs/guides/vision)
- [Documentação - Whisper API](https://platform.openai.com/docs/guides/speech-to-text)
- [JSON Schema Validation](https://json-schema.org/)
